In [ ]:
import math

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K

In [ ]:
# !python3.10 -m pip install seaborn

In [ ]:
# data = pd.read_csv('content/Almond.csv')

# imputer = SimpleImputer(strategy='mean')

# numeric_columns = data.drop(columns=['Type']).columns
# data[numeric_columns] = imputer.fit_transform(data[numeric_columns])

# label_encoder = LabelEncoder()
# data['Type'] = label_encoder.fit_transform(data['Type'])

# scaler = StandardScaler()
# data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# X = data.drop(columns=['Type'])
# y = data['Type']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train.head(), y_train.head()

In [ ]:
data = pd.read_csv('content/Almond.csv')
data = data.drop(columns = ['Unnamed: 0'])

imputer = SimpleImputer(strategy='mean')

numeric_columns = data.drop(columns=['Type']).columns
base_features = ['Length (major axis)', 'Width (minor axis)', 'Thickness (depth)']
derived_features = ['Roundness', 'Aspect Ratio', 'Eccentricity']

data[base_features] = imputer.fit_transform(data[base_features])

data['Roundness'] = (4 * data['Area']) / (math.pi * (data['Length (major axis)'] ** 2))
data['Aspect Ratio'] = (data['Length (major axis)']) / (data['Width (minor axis)'])
data['Eccentricity'] = np.sqrt(np.clip(1 - (data['Width (minor axis)'] / data['Length (major axis)']) ** 2, 0, None))

label_encoder = LabelEncoder()
data['Type'] = label_encoder.fit_transform(data['Type'])

scaler = StandardScaler()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

X = data.drop(columns=['Type'])
y = data['Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head(), y_train.head()

In [ ]:
X.isnull().sum()

In [ ]:
def create_grid_search_model(learning_rate, neurons):
    model = Sequential()
    
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(model=create_grid_search_model, learning_rate=0.001, neurons=32, verbose=0)

param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'neurons': [16, 32, 64]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)

grid_result = grid.fit(X_train, y_train)

print(f"Best Score: {grid_result.best_score_} using {grid_result.best_params_}")

best_model = grid_result.best_estimator_
best_test_acc = best_model.score(X_test, y_test)
print(f'Test accuracy with best hyperparameters: {best_test_acc}')

In [ ]:
scores = grid_result.cv_results_['mean_test_score'].reshape(len(param_grid['learning_rate']), len(param_grid['neurons']))

plt.figure(figsize=(8, 6))
sns.heatmap(scores, annot=True, xticklabels=param_grid['neurons'], yticklabels=param_grid['learning_rate'])
plt.title('Hyperparameter Optimization: Learning Rate vs Neurons')
plt.xlabel('Neurons')
plt.ylabel('Learning Rate')
plt.show()

In [ ]:
std_scores = grid_result.cv_results_['std_test_score'].reshape(len(param_grid['learning_rate']), len(param_grid['neurons']))
mean_scores = grid_result.cv_results_['mean_test_score'].reshape(len(param_grid['learning_rate']), len(param_grid['neurons']))

for i, lr in enumerate(param_grid['learning_rate']):
    for j, neuron in enumerate(param_grid['neurons']):
        print(f"Learning Rate: {lr}, Neurons: {neuron}, Mean Accuracy: {mean_scores[i][j]:.3f}, Std Dev: {std_scores[i][j]:.3f}")

In [ ]:
def create_model_with_optimizer(optimizer):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer=optimizer, 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [ ]:
rprop_optimizer = RMSprop()
adam_optimizer = Adam()

rprop_model = create_model_with_optimizer(rprop_optimizer)
rprop_history = rprop_model.fit(X_train, y_train, 
                                epochs=20, 
                                batch_size=32, 
                                validation_data=(X_test, y_test), 
                                verbose=0)

adam_model = create_model_with_optimizer(adam_optimizer)
adam_history = adam_model.fit(X_train, y_train, 
                              epochs=20, 
                              batch_size=32, 
                              validation_data=(X_test, y_test), 
                              verbose=0)

rprop_loss, rprop_acc = rprop_model.evaluate(X_test, y_test, verbose=0)
adam_loss, adam_acc = adam_model.evaluate(X_test, y_test, verbose=0)

print(f'RProp Test Accuracy: {rprop_acc}')
print(f'RProp Test Loss: {rprop_loss}')
print(f'Adam Test Accuracy: {adam_acc}')
print(f'Adam Test Loss: {adam_loss}')

In [ ]:
def hybrid_model_train(model, X_train, y_train, epochs=20, batch_size=32):
    rprop_optimizer = RMSprop()
    adam_optimizer = Adam()
    
    adam_model = create_model_with_optimizer(adam_optimizer)
    rprop_model = create_model_with_optimizer(rprop_optimizer)
    
    hybrid_train_losses = []
    hybrid_val_losses = []
    hybrid_train_acc = []
    hybrid_val_acc = []
    
    for epoch in range(epochs):
        print(f"Epoch {epoch}")
        adam_updates = []
        rprop_updates = []
        
        for batch in range(0, len(X_train), batch_size):
            print(f"Batch {batch}")
            X_batch = X_train[batch:batch+batch_size]
            y_batch = y_train[batch:batch+batch_size]
            
            adam_model.train_on_batch(X_batch, y_batch)
            adam_updates.append(adam_model.get_weights())
            
            rprop_model.train_on_batch(X_batch, y_batch)
            rprop_updates.append(rprop_model.get_weights())

        print('Getting avg.')
        avg_updates = []
        for adam_w, rprop_w in zip(adam_updates[-1], rprop_updates[-1]):
            avg_w = np.mean([adam_w, rprop_w], axis=0)
            avg_updates.append(avg_w)
        
        print("Applying avg")
        model.set_weights(avg_updates)
        
        print("Calculating Acc and Loss")
        train_loss, train_acc = hybrid_model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = hybrid_model.evaluate(X_test, y_test, verbose=0)
        hybrid_train_acc.append(train_acc)
        hybrid_val_acc.append(val_acc)
        hybrid_train_losses.append(train_loss)
        hybrid_val_losses.append(val_loss)
        
    return hybrid_train_losses, hybrid_val_losses, hybrid_train_acc, hybrid_val_acc

In [ ]:
hybrid_model = create_model_with_optimizer(Adam())

hybrid_train_losses, hybrid_val_losses, hybrid_train_acc, hybrid_val_acc = hybrid_model_train(hybrid_model, X_train, y_train)

In [ ]:
hybrid_val_acc

In [ ]:
hybrid_val_losses

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(adam_history.history['loss'], label=f'Training Loss (Adam)')
plt.plot(adam_history.history['val_loss'], label=f'Validation Loss (Adam)')

plt.plot(rprop_history.history['loss'], label=f'Training Loss (RProp)')
plt.plot(rprop_history.history['val_loss'], label=f'Validation Loss (RProp)')

plt.plot(hybrid_train_losses, label=f'Training Loss (Hybrid)')
plt.plot(hybrid_val_losses, label=f'Validation Loss (Hybrid)')

# Add titles and labels
plt.title('Run 3: Training and Validation Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Show the plot
plt.show()